In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [9]:
def log_progress(message):
    """This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing"""
    with open('CODE1.txt', 'a') as f:
        f.write(f'{datetime.now()}: {message}\n')
    

## Extract from Pakwheels to hit API 5 PAGES OF USED CARS

In [14]:
pip install selenium pandas webdriver-manager


   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.7 MB 1.9 MB/s eta 0:00:05
   ------ --------------------------------- 1.6/9.7 MB 3.7 MB/s eta 0:00:03
   ------------------- -------------------- 4.7/9.7 MB 8.0 MB/s eta 0:00:01
   ------------------------------------- -- 9.2/9.7 MB 11.4 MB/s eta 0:00:01
   -------------------------------------- - 9.4/9.7 MB 9.9 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 9.3 MB/s  0:00:01

   --- ------------------------------------  1/11 [wsproto]
   --- ------------------------------------  1/11 [wsproto]
  Attempting uninstall: typing_extensions
   --- ------------------------------------  1/11 [wsproto]
    Found existing installation: typing_extensions 4.14.1
   --- ------------------------------------  1/11 [wsproto]
    Uninstalling typing_extensions-4.14.1:
   --- -----------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
pip install --upgrade selenium webdriver-manager


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.google.com")
print(driver.title)
driver.quit()


Google


In [35]:
import requests
from bs4 import BeautifulSoup
import json
import csv

all_cars = []

# Loop through pages 1 to 5
for page in range(1, 6):
    url = f"https://www.pakwheels.com/used-cars/search/-/?page={page}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }

    print(f"Fetching Page {page}...")
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # Find all JSON-LD scripts
    scripts = soup.find_all("script", type="application/ld+json")

    for script in scripts:
        try:
            data = json.loads(script.string)
            # Make sure it is a Product listing
            if isinstance(data, dict) and data.get("@type") == ["Product"]:
                title = data.get("name")
                model = data.get("manufacturer")
                year = data.get("modelDate")
                price = data.get("offers", {}).get("price")
                # Extract city from title or description
                city = None
                if title and " in " in title:
                    city = title.split(" in ")[-1]

                car = {
                    "title": title,
                    "model": model,
                    "year": year,
                    "price": price,
                    "city": city
                }
                all_cars.append(car)
        except Exception as e:
            # Skip any malformed JSON
            continue

# Save raw JSON for backup
with open("pakwheels_used_cars_raw.json", "w", encoding="utf-8") as f:
    json.dump(all_cars, f, ensure_ascii=False, indent=4)

# Save clean CSV
csv_columns = ["title", "model", "year", "price", "city"]
with open("pakwheels_used_cars.csv", "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader()
    for car in all_cars:
        writer.writerow(car)

print(f"\nScraping completed! Total cars fetched: {len(all_cars)}")


Fetching Page 1...
Fetching Page 2...
Fetching Page 3...
Fetching Page 4...
Fetching Page 5...

Scraping completed! Total cars fetched: 181


In [36]:
import pandas as pd

df = pd.read_csv("pakwheels_used_cars.csv")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(df)


                                                 title       model  year  \
0                Toyota Prius 2013 for sale in Karachi      Toyota  2013   
1            Suzuki Mehran 2015 for sale in Rawalpindi      Suzuki  2015   
2                 Toyota Hilux 2021 for sale in Multan      Toyota  2021   
3                 KIA Sorento 2022 for sale in Karachi         KIA  2022   
4                  Honda Civic 2008 for sale in Lahore       Honda  2008   
5                  Honda City 2019 for sale in Karachi       Honda  2019   
6                 Toyota Raize 2020 for sale in Lahore      Toyota  2020   
7                  Haval H6 2023 for sale in Islamabad       Haval  2023   
8                   Honda City 2010 for sale in Multan       Honda  2010   
9            Hyundai Santa Fe 2024 for sale in Karachi     Hyundai  2024   
10             Toyota Fortuner 2021 for sale in Lahore      Toyota  2021   
11              Toyota Corolla 2012 for sale in Lahore      Toyota  2012   
12          

## Load to csv

In [37]:
import pandas as pd

# Load CSV
df = pd.read_csv("pakwheels_used_cars.csv")

# Display as table in console
from tabulate import tabulate
print(tabulate(df, headers='keys', tablefmt='grid'))


+-----+---------------------------------------------------+------------+--------+----------+-------------+
|     | title                                             | model      |   year |    price | city        |
+=====+===================================================+============+========+==========+=============+
|   0 | Toyota Prius 2013 for sale in Karachi             | Toyota     |   2013 |  3900000 | Karachi     |
+-----+---------------------------------------------------+------------+--------+----------+-------------+
|   1 | Suzuki Mehran 2015 for sale in Rawalpindi         | Suzuki     |   2015 |  1290000 | Rawalpindi  |
+-----+---------------------------------------------------+------------+--------+----------+-------------+
|   2 | Toyota Hilux 2021 for sale in Multan              | Toyota     |   2021 | 11900000 | Multan      |
+-----+---------------------------------------------------+------------+--------+----------+-------------+
|   3 | KIA Sorento 2022 for sale in 

## Transform to load in CSV